Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

![Credit card being held in hand](credit_card.jpg)

You have been provided with a small subset of the credit card applications a bank receives. The dataset has been loaded as a Pandas DataFrame for you. You will start from there. 

In [175]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [176]:
# Exploratory Data Analysis

print(cc_apps.describe())
cc_apps = cc_apps.drop([11, 13], axis=1) #not necessary features

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


In [177]:
# Split train and test data

cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

In [178]:
# Impute NaN

cc_apps_train_nans_replaced = cc_apps_train.replace('?', np.nan)
cc_apps_test_nans_replaced = cc_apps_test.replace('?', np.nan)

In [179]:
cc_apps_train_nans_replaced[1] = cc_apps_train_nans_replaced[1].astype('float')
cc_apps_test_nans_replaced[1] = cc_apps_test_nans_replaced[1].astype('float')

In [180]:
# Filling NaNs

# Select only numeric columns
numeric_columns = cc_apps_train_nans_replaced.select_dtypes(include=['number'])

# Fill NaN values in numeric columns with their means
cc_apps_train_imputed = cc_apps_train_nans_replaced.copy()
cc_apps_test_imputed = cc_apps_test_nans_replaced.copy()

cc_apps_train_imputed[numeric_columns.columns] = cc_apps_train_nans_replaced[numeric_columns.columns].fillna(cc_apps_train_nans_replaced[numeric_columns.columns].mean())

cc_apps_test_imputed[numeric_columns.columns] = cc_apps_test_nans_replaced[numeric_columns.columns].fillna(cc_apps_train_nans_replaced[numeric_columns.columns].mean())

In [181]:
for col in cc_apps_train_imputed.columns:
    
    if cc_apps_train_imputed[col].dtype == object:
        imputing_value = cc_apps_train_imputed[col].value_counts(ascending=False).iloc[0]
        #print(imputing_value, 'next')
        cc_apps_train_imputed[col] = cc_apps_train_imputed[col].fillna(imputing_value, axis=0)
        cc_apps_test_imputed[col] = cc_apps_test_imputed[col].fillna(imputing_value, axis=0)
        

In [182]:
# Encoding categorical features

cc_apps_train_cat_encoding = pd.get_dummies(cc_apps_train_imputed)
cc_apps_test_cat_encoding = pd.get_dummies(cc_apps_test_imputed)

cc_apps_test_cat_encoding = cc_apps_test_cat_encoding.reindex(columns=cc_apps_train_cat_encoding.columns, fill_value=0)

print(cc_apps_test_cat_encoding.columns, cc_apps_train_cat_encoding.columns)

Index([      1,       2,       7,      10,      14, '0_314',   '0_a',   '0_b',
       '3_341',   '3_l',   '3_u',   '3_y', '4_341',   '4_g',  '4_gg',   '4_p',
        '5_91',  '5_aa',   '5_c',  '5_cc',   '5_d',   '5_e',  '5_ff',   '5_i',
         '5_j',   '5_k',   '5_m',   '5_q',   '5_r',   '5_w',   '5_x', '6_261',
        '6_bb',  '6_dd',  '6_ff',   '6_h',   '6_j',   '6_n',   '6_o',   '6_v',
         '6_z',   '8_f',   '8_t',   '9_f',   '9_t',  '12_g',  '12_p',  '12_s',
        '15_+',  '15_-'],
      dtype='object') Index([      1,       2,       7,      10,      14, '0_314',   '0_a',   '0_b',
       '3_341',   '3_l',   '3_u',   '3_y', '4_341',   '4_g',  '4_gg',   '4_p',
        '5_91',  '5_aa',   '5_c',  '5_cc',   '5_d',   '5_e',  '5_ff',   '5_i',
         '5_j',   '5_k',   '5_m',   '5_q',   '5_r',   '5_w',   '5_x', '6_261',
        '6_bb',  '6_dd',  '6_ff',   '6_h',   '6_j',   '6_n',   '6_o',   '6_v',
         '6_z',   '8_f',   '8_t',   '9_f',   '9_t',  '12_g',  '12_p',  '12_s',
    

In [183]:
print(cc_apps_train_cat_encoding.shape, cc_apps_test_cat_encoding.shape)

(462, 50) (228, 50)


In [184]:
# Train and test data

X_train,y_train = cc_apps_train_cat_encoding.iloc[:,:-1], cc_apps_train_cat_encoding.iloc[:,[-1]]
X_test,y_test = cc_apps_test_cat_encoding.iloc[:,:-1], cc_apps_test_cat_encoding.iloc[:,[-1]]

In [185]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [186]:
print( X_train.shape, X_test.shape)

(462, 49) (228, 49)


In [187]:
#Scaling

scaler = MinMaxScaler(feature_range=(0,1))

rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [188]:
# Train and evaluate logistic regresion model

logreg = LogisticRegression()

logreg.fit(rescaledX_train, y_train)

y_pred = logreg.predict(rescaledX_test)

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[103   0]
 [  0 125]]


In [189]:
# Hyperparameter tuning

tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

param_grid = {'tol': tol,
             'max_iter': max_iter}

grid_model = GridSearchCV(logreg, param_grid=param_grid, cv = 5)

grid_model_result = grid_model.fit(rescaledX_train, y_train)

best_model = grid_model_result.best_estimator_
best_params = grid_model_result.best_params_
best_score = grid_model_result.best_score_

print(best_model, best_params)

LogisticRegression(tol=0.01) {'max_iter': 100, 'tol': 0.01}


In [190]:
# Evaluate best model in test set

y_pred_gm = best_model.predict(rescaledX_test)

cm_gm = confusion_matrix(y_test, y_pred_gm)

print(cm_gm)

[[103   0]
 [  0 125]]
